In [1]:
import pandas as pd
import jutil

In [112]:
sql = "select DISTINCT(m1.student_id),m1.course_code as code1,m2.course_code as code2,m1.course_name as name1,m2.course_name as name2,m1.pmark as mark1,m2.pmark as mark2, \
m1.speciality_code from view_stu_course_mark m1 join view_stu_course_mark m2 on m1.student_id=m2.student_id \
where m1.course_code='6011310' and m2.course_code='6011320'"
df = jutil.load_pd_df(sql)
df.head()

,code1,code2,mark1,mark2,name1,name2,speciality_code,student_id
0,6011310,6011320,35.0,67.0,高等数学Ⅰ,高等数学Ⅱ,0305,19970936
1,6011310,6011320,65.0,62.0,高等数学Ⅰ,高等数学Ⅱ,0101,19980016
2,6011310,6011320,60.0,28.0,高等数学Ⅰ,高等数学Ⅱ,0101,19980030
3,6011310,6011320,60.0,23.0,高等数学Ⅰ,高等数学Ⅱ,0101,19980173
4,6011310,6011320,67.0,71.0,高等数学Ⅰ,高等数学Ⅱ,0109,19980386


In [120]:
# 求每门课成绩的平均值

mark_mean1 = float(df.agg({'mark1':'mean'}))
mark_mean2 = float(df.agg({'mark2':'mean'}))
print(mark_mean1)
print(mark_mean2)

67.90256653423044
60.21227064907062


In [119]:
type(df.agg({'mark1':'mean'}))

pandas.core.series.Series

In [104]:
dist1 = (100-mark_mean1)/2 
dist2 = (100-mark_mean2)/2 
# 求优秀的分数线
good_mark1 = mark_mean1+ dist1
good_mark2 = mark_mean2+ dist2
print(good_mark1)
print(good_mark2)

# 求差的分数线
def bad_mark(mark_mean,dist):
    line = mark_mean - dist
    return line if line>60 else 60.0
bad_mark1 = bad_mark(mark_mean1,dist1)
bad_mark2 = bad_mark(mark_mean2,dist2)
print(bad_mark1)
print(bad_mark2)

92.68692512184316
88.97126894107222
78.06077536552945
66.91380682321667


In [105]:
# 过滤出优秀和差的学生数据集

stu_goods1 = df[df.mark1>good_mark1]
stu_goods2 = df[df.mark2>good_mark2]
stu_all_goods = stu_goods1[stu_goods1.mark2>good_mark2]

stu_bads1 = df[df.mark1<bad_mark1]
stu_bads2 = df[df.mark2<bad_mark2]
stu_all_bads = stu_bads1[stu_bads1.mark2<bad_mark2]

temp = df[df.mark1<=good_mark1]
stu_normal1 = temp[temp.mark2>=bad_mark1]
temp = stu_normal1[stu_normal1.mark2>=bad_mark2]
stu_all_normal = temp[temp.mark2<=good_mark2]

In [106]:
goods2_cnt = stu_goods2.count()['mark1']
all_goods_cnt = stu_all_goods.count()['mark1']
good_prob = all_goods_cnt /goods2_cnt
good_prob

0.061297860866438972

In [107]:
bads2_cnt = stu_bads2.count()['mark2']
all_bads_cnt = stu_all_bads.count()['mark2']
bad_prob = all_bads_cnt/bads2_cnt
bad_prob

0.027107536490914508

In [108]:
# 正常中的比例
all_cnt = df.count()['student_id']
normal2_cnt =  all_cnt - goods2_cnt - bads2_cnt
all_normal_cnt = stu_all_normal.count()['mark2']
normal_prob = all_normal_cnt/normal2_cnt
normal_prob

0.54740871613663133

In [109]:
print(all_cnt)
print(goods2_cnt)
print(all_goods_cnt)
print(bads2_cnt)
print(all_bads_cnt)
print(normal2_cnt)
print(all_normal_cnt)

56425
5563
341
6714
182
44148
24167


In [110]:
# 最后按照比例分配权重
cnt = goods2_cnt + bads2_cnt
w_good = goods2_cnt / cnt
w_bad = bads2_cnt / cnt
# w_normal = normal2_cnt / all_cnt
final_prob = w_good*good_prob+w_bad*bad_prob #+w_normal*normal_prob
final_prob

0.042599983709375254

In [111]:
# 最后按照比例分配权重
w_good = goods2_cnt / all_cnt
w_bad = bads2_cnt / all_cnt
w_normal = normal2_cnt / all_cnt
final_prob = w_good * good_prob + w_bad * bad_prob + w_normal * normal_prob
final_prob

0.4375719982277359